# consolidated data

In [1]:
import matplotlib.pyplot as plt
import numpy as np
import xarray as xr
%matplotlib inline
%config InlineBackend.figure_format='retina'
import cmocean
import cartopy.crs as ccrs
import matplotlib.path as mpath
import nc_time_axis
import pandas as pd
import matplotlib as mpl
import momlevel as ml
import gcsfs
import zarr
from matplotlib.ticker import FormatStrFormatter

In [2]:
mask_CM4= xr.open_dataset('/home/pachamama/shared/antarctica_mask/CM4_mask_antarctica.nc')
mask_ESM4= xr.open_dataset('/home/pachamama/shared/antarctica_mask/ESM4_mask_antarctica.nc')
mask_ERA5= xr.open_dataset('/home/pachamama/shared/antarctica_mask/ERA5_mask_antarctica.nc')

In [3]:
##importing Eastward Near-Surface Wind data from google cloud. Amon
df_uas = pd.read_csv('https://storage.googleapis.com/cmip6/cmip6-zarr-consolidated-stores.csv')
df_uas	
# step two … # replace with the id you need and other info to ween down datasets 
df_u = df_uas.query("source_id=='GFDL-CM4' & experiment_id == 'piControl' & variable_id == 'uas'& table_id == 'Amon'")
df_u
# step three - loading in the data and storing it
gcs = gcsfs.GCSFileSystem(token='anon')
zstore = df_u.zstore.values[-1]
mapper = gcs.get_mapper(zstore)
ds_uas = xr.open_zarr(mapper, consolidated=True)

##importing Northward Near-Surface Wind data from google cloud.
df_vas = pd.read_csv('https://storage.googleapis.com/cmip6/cmip6-zarr-consolidated-stores.csv')
df_vas	
# step two … # replace with the id you need and other info to ween down datasets 
df_v = df_vas.query("source_id=='GFDL-CM4' & experiment_id == 'piControl' & variable_id == 'vas'& table_id == 'Amon'")
df_v
# step three 
gcs = gcsfs.GCSFileSystem(token='anon')
zstore = df_v.zstore.values[-1]
mapper = gcs.get_mapper(zstore)
ds_vas = xr.open_zarr(mapper, consolidated=True)

## renaming the variables for the merge to get the wind speed.
Uas = ds_uas.uas
Vas = ds_vas.vas
Uas

## selecting the UAS variable and renaming it to Uas_winds.
## This allows us to look at the value we want for eastward wind.
Uas_winds = ds_uas.uas
## Slicing the eastward wind into the south pole region, as well as averaging the 500 year time values.
## This provides the average for 500 year. Can be named whatever I just named it this bc im cool.
Uas_151_650 = Uas_winds.sel(lat=slice(-89.5,-55)).sel(time=slice('0151-01-16','0650-12-16')).mean(dim='time')
Uas_151_650

<xarray.DataArray 'uas' (lat: 35, lon: 288)>
dask.array<mean_agg-aggregate, shape=(35, 288), dtype=float32, chunksize=(35, 288), chunktype=numpy.ndarray>
Coordinates:
    height   float64 ...
  * lat      (lat) float64 -89.5 -88.5 -87.5 -86.5 ... -58.5 -57.5 -56.5 -55.5
  * lon      (lon) float64 0.625 1.875 3.125 4.375 ... 355.6 356.9 358.1 359.4

In [4]:
##UAS wind dataset defined above
Uas_winds
## slicing the UAS into south polar lat
Antsliceuas = Uas_winds.sel(lat=slice(-89.5,-55))
Antsliceuas
## grouping the data into seasons and averaging the 500 seasons
uas_seasonal = Antsliceuas.groupby('time.season').mean(dim='time')
uas_seasonal
## creating seasonal variables
uas_summer = uas_seasonal.sel(season='DJF') ##southern hemisphere has summer in december,jan,feb
uas_winter = uas_seasonal.sel(season='JJA') ### winter months in southern hemisphere
uas_spring = uas_seasonal.sel(season='SON') ## spring months in southern hemisphere
uas_fall = uas_seasonal.sel(season='MAM') ## fall months in southern hemisphere

## selecting the Vas variable and naming it to Vas_winds. allows us to look at northward winds.
Vas_winds = ds_vas.vas
## slicing Vas_winds to south polar region and averaging 500 year to get average.
Vas_151_650 = Vas_winds.sel(lat=slice(-89.5,-55)).sel(time=slice('0151-01-16','0650-12-16')).mean(dim='time')
Vas_151_650

## VAS data set as defined earlier
Vas_winds
## slicing vas into south pole lat
antslicevas = Vas_winds.sel(lat=slice(-89.5,-55))
## grouping Vas into seasons and averaging the 500 seasons.
vas_seasonal = antslicevas.groupby('time.season').mean(dim='time')
## plotting to see DJF / month values
vas_seasonal
## creating seasonal variables ## im not sure we need these actually...
vas_summer = vas_seasonal.sel(season='DJF') ##southern hemisphere has summer in december,jan,feb
vas_winter = vas_seasonal.sel(season='JJA') ### winter months in southern hemisphere
vas_spring = vas_seasonal.sel(season='SON') ## spring months in southern hemisphere
vas_fall = vas_seasonal.sel(season='MAM') ## fall months in southern hemisphere

In [5]:
## merge winds by merging eastward and northward.
mergewinds = xr.merge([Uas,Vas]) # merge winds to make it easy
xmerge = mergewinds.lon #extracting the lon component fomr the mergewinds array
ymerge = mergewinds.lat
umerge = mergewinds.uas 
vmerge = mergewinds.vas
speedmerge = np.sqrt(np.square(umerge) + np.square(vmerge)) ## calculating speed using vector sum formula
## average
timeaveragemerge = mergewinds.sel(time=slice('0151-01-16','0650-12-16')).mean(dim='time')
timeaveragemerge

### slicing the speed for the south pole
speedslice = speedmerge.sel(lat=slice(-89.5,-55)).sel(time=slice('0151-01-16','0650-12-16')).mean(dim='time')
## UAS seasonal dataset
uas_seasonal
## VAS seasonal plot
vas_seasonal
## merging the UAS seasonal and Vas seasonal to get merge seasonal
seasonalmerge = xr.merge([uas_seasonal,vas_seasonal])
seasonalmerge

<xarray.Dataset>
Dimensions:  (lat: 35, lon: 288, season: 4)
Coordinates:
    height   float64 10.0
  * lat      (lat) float64 -89.5 -88.5 -87.5 -86.5 ... -58.5 -57.5 -56.5 -55.5
  * lon      (lon) float64 0.625 1.875 3.125 4.375 ... 355.6 356.9 358.1 359.4
  * season   (season) object 'DJF' 'JJA' 'MAM' 'SON'
Data variables:
    uas      (season, lat, lon) float32 dask.array<chunksize=(1, 35, 288), meta=np.ndarray>
    vas      (season, lat, lon) float32 dask.array<chunksize=(1, 35, 288), meta=np.ndarray>
Attributes:
    cell_measures:  area: areacella
    cell_methods:   area: time: mean
    interp_method:  conserve_order2
    long_name:      Eastward Near-Surface Wind
    original_name:  uas
    standard_name:  eastward_wind
    units:          m s-1

In [6]:
## calculating the seasonal wind speed
seasonalspeedmerge = np.sqrt(np.square(seasonalmerge.uas) + np.square(seasonalmerge.vas))
## slicing the summer speed
Summer_szn_speed = seasonalspeedmerge.sel(season=slice('DJF')).mean(dim='season')
## Slicing summer season from seasonalmerge
Summer_szn= seasonalmerge.sel(season='DJF')
## slicing seasonalspeed to get speed for winter and averaging it
Winter_szn_speed = seasonalspeedmerge.sel(season=slice('JJA')).mean(dim='season')
Winter_szn= seasonalmerge.sel(season='JJA')
## FALL seasonal speed merge and averaging
Fall_szn_speed = seasonalspeedmerge.sel(season=slice('MAM')).mean(dim='season')
Fall_szn= seasonalmerge.sel(season='MAM')
## slicing seasonalspeed to get spring values and averaging it
Spring_szn_speed = seasonalspeedmerge.sel(season=slice('SON')).mean(dim='season')
Spring_szn= seasonalmerge.sel(season='SON')

# ESM4

In [7]:
google_cloud_cmip6 = pd.read_csv('https://storage.googleapis.com/cmip6/cmip6-zarr-consolidated-stores.csv')
#defines the Eastward Near-Surface Wind
gfdl_esm4_uas_data = google_cloud_cmip6.query(
    "table_id == 'Amon' & \
    variable_id == 'uas' & \
    experiment_id == 'piControl' & \
    institution_id == 'NOAA-GFDL' &  \
    source_id == 'GFDL-ESM4'")

gcs = gcsfs.GCSFileSystem(token='anon')
zstore_esm4_uas = gfdl_esm4_uas_data.zstore.values[-1]
mapper_esm4_uas = gcs.get_mapper(zstore_esm4_uas)

esm4_uas = xr.open_zarr(mapper_esm4_uas, 
                                consolidated=True,use_cftime=True).uas.rename({'lat':'latitude',
                                                             'lon':'longitude'})

#defines the NorthWard Near-Surface Wind
gfdl_esm4_vas_data = google_cloud_cmip6.query(
    "table_id == 'Amon' & \
    variable_id == 'vas' & \
    experiment_id == 'piControl' & \
    institution_id == 'NOAA-GFDL' &  \
    source_id == 'GFDL-ESM4'")

zstore_esm4_vas = gfdl_esm4_vas_data.zstore.values[-1]
mapper_esm4_vas = gcs.get_mapper(zstore_esm4_vas)

esm4_vas = xr.open_zarr(mapper_esm4_vas, 
                                consolidated=True,use_cftime=True).vas.rename({'lat':'latitude',
                                                             'lon':'longitude'})
uase4=esm4_uas#.sel(time=slice('0001-01-16','0500-01-16')).mean(dim='time')#.groupby('time.season').mean(dim='time')
vase4 = esm4_vas#.sel(time=slice('0001-01-16','0500-01-16')).mean(dim='time')

## Slicing the eastward wind into the south pole region, as well as averaging the 500 year time values.
## This provides the average for 500 year. Can be named whatever I just named it this bc im cool.
uase4_500_avg=uase4.sel(latitude=slice(-89.5,-55)).sel(time=slice('0151-01-16','0650-12-16')).mean(dim='time')

In [8]:
uase4 #eastward wind variable
## slicing the uase4 into south polar lat
ESM4_Antsliceuas = uase4.sel(latitude=slice(-89.5,-55))
ESM4_Antsliceuas
## grouping the data into seasons and averaging the 500 seasons
ESM4_uas_seasonal = ESM4_Antsliceuas.groupby('time.season').mean(dim='time')
ESM4_uas_seasonal
## grouping into seasonal varibles 
ESM4_uas_summer = ESM4_uas_seasonal.sel(season='DJF') ##southern hemisphere has summer in december,jan,feb
ESM4_uas_winter = ESM4_uas_seasonal.sel(season='JJA') ### winter months in southern hemisphere
ESM4_uas_spring = ESM4_uas_seasonal.sel(season='SON') ## spring months in southern hemisphere
ESM4_uas_fall = ESM4_uas_seasonal.sel(season='MAM') ## fall months in southern hemisphere

## Slicing the eastward wind into the south pole region, as well as averaging the 500 year time values.
## This provides the average for 500 year. Can be named whatever I just named it this bc im cool.
vase4_500_avg=vase4.sel(latitude=slice(-89.5,-55)).sel(time=slice('0151-01-16','0650-12-16')).mean(dim='time')
vase4 #northward wind variable
## slicing the vase4 into south polar lat
ESM4_Antslicevas = vase4.sel(latitude=slice(-89.5,-55))
ESM4_Antslicevas
## grouping the data into seasons and averaging the 500 seasons
ESM4_vas_seasonal = ESM4_Antslicevas.groupby('time.season').mean(dim='time')
ESM4_vas_seasonal
## creating seasonal variables 
ESM4_vas_summer = ESM4_vas_seasonal.sel(season='DJF') ##southern hemisphere has summer in december,jan,feb
ESM4_vas_winter = ESM4_vas_seasonal.sel(season='JJA') ### winter months in southern hemisphere
ESM4_vas_spring = ESM4_vas_seasonal.sel(season='SON') ## spring months in southern hemisphere
ESM4_vas_fall = ESM4_vas_seasonal.sel(season='MAM') ## fall months in southern hemisphere

In [9]:
uase4=esm4_uas#.sel(time=slice('0001-01-16','0500-01-16')).mean(dim='time')#.groupby('time.season').mean(dim='time')
vase4 = esm4_vas#.sel(time=slice('0001-01-16','0500-01-16')).mean(dim='time')
mergedwindse4 = xr.merge([uase4,vase4])
esm4_xmerge=mergedwindse4.longitude
esm4_ymerge=mergedwindse4.latitude
esm4_umerge=mergedwindse4.uas
esm4_vmerge=mergedwindse4.vas
esm4_speed_merge=np.sqrt(np.square(esm4_umerge) + np.square(esm4_vmerge)).sel(latitude=slice(-90,-55))
esm4_speed_merge
#wind speed around antarctica 
#####
esm4_timeaveragemerge = mergedwindse4.sel(time=slice('0151-01-16','0650-12-16')).mean(dim='time')
esm4_timeaveragemerge

### slicing the speed for the south pole
ESM4_speedslice = esm4_speed_merge.sel(latitude=slice(-89.5,-55)).sel(time=slice('0151-01-16','0650-12-16')).mean(dim='time')
ESM4_speedslice
##
ESM4_uas_seasonal #defined earlier
ESM4_vas_seasonal #defined earlier
## merging the UAS seasonal and Vas seasonal to get merge seasonal
esm4_seasonalmerge = xr.merge([ESM4_uas_seasonal,ESM4_vas_seasonal])
esm4_seasonalmerge
##
## calculating the seasonal wind speed
esm4_SPEED_seasonalmerge = np.sqrt(np.square(esm4_seasonalmerge.uas) + np.square(esm4_seasonalmerge.vas))
esm4_SPEED_seasonalmerge
## merging the UAS seasonal and Vas seasonal to get merge seasonal
esm4_seasonalmerge = xr.merge([ESM4_uas_seasonal,ESM4_vas_seasonal])
esm4_seasonalmerge
#this has the same variables

<xarray.Dataset>
Dimensions:    (latitude: 35, longitude: 288, season: 4)
Coordinates:
    height     float64 10.0
  * latitude   (latitude) float64 -89.5 -88.5 -87.5 -86.5 ... -57.5 -56.5 -55.5
  * longitude  (longitude) float64 0.625 1.875 3.125 4.375 ... 356.9 358.1 359.4
  * season     (season) object 'DJF' 'JJA' 'MAM' 'SON'
Data variables:
    uas        (season, latitude, longitude) float32 dask.array<chunksize=(1, 35, 288), meta=np.ndarray>
    vas        (season, latitude, longitude) float32 dask.array<chunksize=(1, 35, 288), meta=np.ndarray>
Attributes:
    cell_measures:  area: areacella
    cell_methods:   area: time: mean
    interp_method:  conserve_order2
    long_name:      Eastward Near-Surface Wind
    original_name:  uas
    standard_name:  eastward_wind
    units:          m s-1

In [10]:
esm4_SPEED_seasonalmerge = np.sqrt(np.square(esm4_seasonalmerge.uas) + np.square(esm4_seasonalmerge.vas))
esm4_SPEED_seasonalmerge
## slicing the summer speed
ESM4_Summer_szn_speed = esm4_SPEED_seasonalmerge.sel(season=slice('DJF')).mean(dim='season')
## Slicing summer season from seasonalmerge
ESM4_Summer_szn= esm4_seasonalmerge.sel(season='DJF') #MUST USE SEASONAL MERGE NOT SPEED SEASONAL MERGE. BE CAREFUL
## slicing the winter speed
ESM4_winter_szn_speed = esm4_SPEED_seasonalmerge.sel(season=slice('JJA')).mean(dim='season')
## Slicing winter season from seasonalmerge
ESM4_winter_szn= esm4_seasonalmerge.sel(season='JJA') #MUST USE SEASONAL MERGE NOT SPEED SEASONAL MERGE. BE CAREFUL
## slicing the fall speed
ESM4_fall_szn_speed = esm4_SPEED_seasonalmerge.sel(season=slice('MAM')).mean(dim='season')
## Slicing fall season from seasonalmerge
ESM4_fall_szn= esm4_seasonalmerge.sel(season='MAM') #MUST USE SEASONAL MERGE NOT SPEED SEASONAL MERGE. BE CAREFUL
## slicing the spring speed
ESM4_spring_szn_speed = esm4_SPEED_seasonalmerge.sel(season=slice('SON')).mean(dim='season')
## Slicing spring season from seasonalmerge
ESM4_spring_szn= esm4_seasonalmerge.sel(season='SON') #MUST USE SEASONAL MERGE NOT SPEED SEASONAL MERGE. BE CAREFUL

In [11]:
#rename coordinates to make things easy and consistent
esm4_speedslice_rename=ESM4_speedslice.rename({'latitude': 'lat', 'longitude': 'lon'}) 
esm4_speedslice_rename

<xarray.DataArray (lat: 35, lon: 288)>
dask.array<mean_agg-aggregate, shape=(35, 288), dtype=float32, chunksize=(35, 288), chunktype=numpy.ndarray>
Coordinates:
    height   float64 10.0
  * lat      (lat) float64 -89.5 -88.5 -87.5 -86.5 ... -58.5 -57.5 -56.5 -55.5
  * lon      (lon) float64 0.625 1.875 3.125 4.375 ... 355.6 356.9 358.1 359.4